In [7]:
pip install trimesh

In [8]:
import os
import random
import glob
import trimesh
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import scipy 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv3D, Dense ,Dropout, Flatten, Activation, MaxPooling3D, Input

from sklearn.model_selection import train_test_split, KFold,StratifiedKFold

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt


In [9]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    print("reading from cloud...")
    path = "/content/drive/MyDrive/Colab Notebooks/MPPE/data_70p_overhang/stls_opt/"
    path_vox = "/content/drive/MyDrive/Colab Notebooks/MPPE/data_70p_overhang/voxelsFloat/"
    
except:
    import pathlib
    print("reading from disk")
    path = str(pathlib.Path().resolve())+"/dataset/stls_opt/"
    path_vox = str(pathlib.Path().resolve())+"/dataset/voxels/"


mesh = trimesh.load(path+"63_8475_10313_17790_28845.stl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
reading from cloud...


In [10]:
def calculate_cost(filename):
  #params
  #time - parts usally need 15 minutes for preparation
  #1h - 10eur
  price_per_h = 2
  price_per_meter = 0.17#0.1133 #150 meters costs 17e 
  filament_diameter = 2.85 #mm
  filament_phi = np.power(filament_diameter/2,2)*np.pi #mm^2
  
  support_removal_multiplier= 2
  support_removal_multiplier_time = 1.5
  profit_margin = 0.1
  #larger parts need longer time + 5 minutes slice time. 
  
  #get data from filename
  data = filename.replace(".","_").split("_")
  data = [int(x) for x in data[:-1]]

  #calculate cost from filament lenght

  #calulate cost no support 
  cost_filament = ((data[2]/(filament_phi*1000)))*price_per_meter

  #calulcate cost of printing supports only 
  cost_supports = (abs(data[2]-data[4])/(filament_phi*1000))*price_per_meter
  cost_supports_removal = support_removal_multiplier*cost_supports

  #calulate timewise cost in hours
  print_time = data[1]
  print_time_supports = data[3]

  #convert to horus and include time for removing supports 
  print_time= (((print_time + 2*abs(print_time-print_time_supports))*support_removal_multiplier_time)/3600)
  print_time_cost = print_time * price_per_h

  #cena
  cost = (cost_filament+cost_supports+cost_supports_removal+print_time_cost)
  cost = (cost*profit_margin)+cost

  return cost

#cost = calculate_cost("89_88123_134991_106833_152824.stl")
#cost = calculate_cost("62_1605_2129_2398_3618.stl")
cost = calculate_cost("108_31956_48635_34435_50936.stl")
print(cost)

35.46582523917991


In [11]:
X = np.zeros([0,100,100,100])
X_volume = np.zeros([0,1])
X_area = np.zeros([0,1])

y = np.zeros(0)
count = 0
for filename in os.listdir(path_vox):
    if filename.endswith(".npy"):
        count = count +1
        #print("appending", filename.split(".")[0])
        img = np.load(path_vox+filename)

        img =(img -(np.min(img)))
        img = img/np.max(img)

        X = np.append(X,img[np.newaxis,...],axis=0)
        y = np.append(y,calculate_cost(filename))
        
        mesh = trimesh.load(path+filename.split(".")[0]+".stl")
        
        mv = np.array(abs(mesh.volume))
        X_volume = np.append(X_volume,mv[np.newaxis,...])

        ma = np.array(abs(mesh.area))
        X_area = np.append(X_area,ma[np.newaxis,...])

print(count)
X_area = X_area[...,np.newaxis]
X_volume = X_volume[...,np.newaxis]
X = X[...,np.newaxis]

194


In [12]:
n_samples = X.shape[0]
train_ind, test_ind  = train_test_split(range(n_samples), test_size=0.2, random_state=36)

X_train = X[train_ind]
X_train_volume = X_volume[train_ind]
X_train_area = X_area[train_ind]
y_train = y[train_ind]

X_test = X[test_ind]
X_test_volume = X_volume[test_ind]
X_test_area = X_area[test_ind]
y_test = y[test_ind]

print("train",X_train.shape,X_train_volume.shape,X_train_area.shape)
print("test",X_test.shape,X_test_volume.shape,X_test_area.shape)

train (155, 100, 100, 100, 1) (155, 1) (155, 1)
test (39, 100, 100, 100, 1) (39, 1) (39, 1)


In [22]:
def augment(img,angle):

  img = scipy.ndimage.rotate(img,angle,axes=(0,1),order=3)
  n = 100
  desiredshape = np.array([n,n,n])
  zoomArray = desiredshape.astype(np.float32) / img.shape
  img = scipy.ndimage.interpolation.zoom(img, zoomArray).astype(np.float32)
  
  img =(img -(np.min(img)))

  img = img/np.max(img)
  return img

In [23]:
X_train_2 = np.zeros([0,100,100,100])
X_train_volume_2 = np.zeros([0,1])
X_train_area_2 = np.zeros([0,1])
y_train_2 = np.zeros(0)

for i,img in enumerate(X_train):
  for angle in range(0,330,30):
    img_aug = augment(img[:,:,:,0],angle)

    X_train_2 = np.append(X_train_2,img_aug[np.newaxis,...],axis=0)
    X_train_volume_2 = np.append(X_train_volume_2,X_train_volume[i])
    X_train_area_2 = np.append(X_train_area_2,X_train_area[i])

    y_train_2 = np.append(y_train_2,y_train[i])
  
X_train_area_2 = X_train_area_2[...,np.newaxis]
X_train_volume_2 = X_train_volume_2[...,np.newaxis]
X_train_2 = X_train_2[...,np.newaxis]

KeyboardInterrupt: ignored

In [ ]:
X_train = X_train_2
X_trian_volume = X_train_2
X_train_area_2 = X_train_area
y_train = y_train_2

In [13]:
#my net
input1 = keras.Input(shape=(X_train.shape[1:]))
input2 = keras.Input(shape=(X_train_volume.shape[1:]))
input3 = keras.Input(shape=(X_train_area.shape[1:]))

#y = Dense(1)(input2)
activ = "relu"
# x = Conv3D(63,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(input1)
# x = Conv3D(64,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
#x = MaxPooling3D(pool_size=(2, 2, 2))(x)
#x = layers.Dropout(0.3)(x)
x = Conv3D(32,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(input1)
x = Conv3D(32,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
#x = layers.Dropout(0.3)(x)
x = Conv3D(16,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = Conv3D(16,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)

x = Conv3D(8,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = Conv3D(8,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = MaxPooling3D(pool_size=(1, 1, 1))(x)

x = Activation('relu')(x)

x = layers.Dropout(0.3)(x)

x = Flatten()(x)

x = layers.Concatenate(axis=1)([x,input2])

#x = Dense(4, activation = 'relu')(x)
x = layers.Concatenate(axis=1)([x,input3])

dense = Dense(2000, activation = activ)(x)
dense = Dense(300, activation = activ)(dense)
dense = Dense(150, activation = activ)(dense)
dense = Dense(20, activation = activ)(dense)
dense = Dense(16, activation = activ)(dense)
    # final layer with 10 neurons to classify the instances
output = Dense(1, activation = 'linear')(dense)
    
#outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=[input1,input2,input3], outputs=output, name="jjnet")

In [ ]:
#@title Default title text
#paper net 
input1 = keras.Input(shape=(X_train.shape[1:]))
input2 = keras.Input(shape=(X_train_volume.shape[1:]))
input3 = keras.Input(shape=(X_train_area.shape[1:]))



x = Conv3D(16,(3,3,3), activation='LeakyReLU', padding = 'same')(input1)
x = Conv3D(16,(3,3,3), activation='LeakyReLU', padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = layers.Dropout(0.3)(x)

x = Conv3D(32,(3,3,3), activation='LeakyReLU', padding = 'same')(x)
x = Conv3D(32,(3,3,3), activation='LeakyReLU', padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = layers.Dropout(0.3)(x)

x = Conv3D(8,(3,3,3), activation='LeakyReLU', padding = 'same')(x)

x = layers.Dropout(0.8)(x)

x = Flatten()(x)

x = layers.Concatenate(axis=1)([x,input2])
x = Dense(8, activation = 'LeakyReLU')(x)
dense = Dense(2000, activation = 'LeakyReLU')(x)
dense = Dense(300, activation = 'LeakyReLU')(dense)
dense = Dense(150, activation = 'LeakyReLU')(dense)
dense = Dense(20, activation = 'LeakyReLU')(dense)
dense = Dense(16, activation = 'LeakyReLU')(dense)
    # final layer with 10 neurons to classify the instances
output = Dense(1, activation = 'linear')(dense)
    
#outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=[input1,input2], outputs=output, name="papernet")

In [14]:
y_train_max = y_train.max()

X_train_volume_max = X_train_volume.max()

model.compile(
    loss="MAE",
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["MAPE"],
)

model.fit([X_train,np.log(X_train_volume), np.log(X_train_area)], np.log(y_train),validation_data = ([X_test,np.log(X_test_volume),np.log(X_test_area)],y_test), batch_size = 8, epochs = 100, verbose = 1)


Epoch 1/100
20/20 [==============================] - 15s 147ms/step - loss: 2.5125 - MAPE: 98.5721 - val_loss: 26.3737 - val_MAPE: 95.9763
Epoch 2/100
20/20 [==============================] - 2s 114ms/step - loss: 2.3296 - MAPE: 107.6855 - val_loss: 26.2187 - val_MAPE: 93.2831
Epoch 3/100
20/20 [==============================] - 2s 111ms/step - loss: 2.2039 - MAPE: 115.3557 - val_loss: 26.0665 - val_MAPE: 90.8007
Epoch 4/100
20/20 [==============================] - 2s 111ms/step - loss: 2.0723 - MAPE: 123.8600 - val_loss: 25.8888 - val_MAPE: 88.2302
Epoch 5/100
20/20 [==============================] - 2s 112ms/step - loss: 1.9322 - MAPE: 128.0267 - val_loss: 25.7299 - val_MAPE: 86.7700
Epoch 6/100
20/20 [==============================] - 2s 113ms/step - loss: 1.8084 - MAPE: 142.5538 - val_loss: 25.5642 - val_MAPE: 85.2534
Epoch 7/100
20/20 [==============================] - 2s 113ms/step - loss: 1.6818 - MAPE: 152.2628 - val_loss: 25.4093 - val_MAPE: 84.2726
Epoch 8/100
20/20 [========

In [15]:
y_train_max = y_train.max()
preds = model.predict([X_test, np.log(X_test_volume),np.log(X_test_area)])#*y_train_max

In [18]:
np.sum(((preds)-(np.log(y_test))))/len(preds)

5.201200635092709

In [19]:
for e,i in zip(np.exp(preds),y_train):
  print("preds",e,"truth",i)

preds [5.840907] truth 44.139400793438085
preds [1.8039908] truth 2.5995062020154145
preds [40.645615] truth 5.754845714363635
preds [15.810597] truth 23.984738485878957
preds [13.847299] truth 35.33793741462762
preds [15.432901] truth 308.1255287978894
preds [23.719334] truth 0.16621017683993575
preds [34.044037] truth 41.26037494139206
preds [10.6041975] truth 38.63736618631828
preds [34.80188] truth 1.0134805649190142
preds [27.365366] truth 34.1665999012935
preds [8.479455] truth 30.83907842221318
preds [1.6437978] truth 0.16715615662385272
preds [23.813326] truth 9.504685065708175
preds [19.133078] truth 18.833951718741364
preds [23.416723] truth 0.41671098084251973
preds [33.0016] truth 18.291609771384866
preds [31.628645] truth 0.562620939482206
preds [28.740927] truth 29.221974055770225
preds [8.696074] truth 6.152599447377891
preds [4.9074836] truth 7.415828151285955
preds [7.870447] truth 0.8954621638458699
preds [8.050213] truth 19.68306807520568
preds [5.832057] truth 62.18